<a href="https://colab.research.google.com/github/jackychencw/MIE1624_Course_Project_Group19/blob/Lawrence-nlp-preprocess/MIE1624_project_train_keras2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import pandas as pd
import numpy as np
import sklearn

In [33]:
# train_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/preproced_train_df.csv")
train_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/preproced_train_df_without_tag.csv")

In [34]:
train_df = train_df.dropna(subset=['preproced_text'])

In [35]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=500)
tokenizer.fit_on_texts(train_df['preproced_text'])
tokenized_text = tokenizer.texts_to_matrix(train_df['preproced_text'], mode='count')

In [36]:
X = np.concatenate([tokenized_text, train_df['polarities'].values.reshape(-1, 1), train_df['subjectivities'].values.reshape(-1, 1)], axis=1)
y = train_df['overall'].values

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=42)

In [52]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import concatenate
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras import Input
from keras import Model
from keras.optimizers.schedules import ExponentialDecay
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras.optimizers import Adam
class ReviewTextRegressor():
  def __init__(self, seq_size, meta_size, output_size, lr, embedding_size, epoch, batch_size, save_path):
    self.seq_size = seq_size
    self.meta_size = meta_size
    self.output_size = output_size
    self.epoch = epoch
    self.batch_size = batch_size
    embedding_vecor_length = 128
    self.model = Sequential()

    # untag 
    nlp_input = Input(shape=(seq_size,)) 
    meta_input = Input(shape=(meta_size,))
    emb = Embedding(output_dim=embedding_vecor_length, input_dim=500, input_length=seq_size)(nlp_input) 
    nlp_out = Bidirectional(LSTM(128))(emb) 
    concat = concatenate([nlp_out, meta_input]) 
    regressor = Dense(32, activation='relu')(concat) 
    output = Dense(1, activation='relu')(regressor) 
    self.model = Model(inputs=[nlp_input , meta_input], outputs=[output])
    lr_schedule = ExponentialDecay(
            lr,
            decay_steps=1874,
            decay_rate=0.9,
            staircase=True)

    opt = Adam(learning_rate=lr_schedule)
    self.model.compile(loss="mean_squared_error", optimizer=opt, metrics=['mse'])
    self.early_stop = EarlyStopping(monitor='loss', min_delta=0.005, patience=5)
    self.check_pt = ModelCheckpoint(filepath=save_path, save_weights_only=True,
                                             verbose=1)
    

    # # tag 0.65
    # nlp_input = Input(shape=(seq_size,)) 
    # meta_input = Input(shape=(meta_size,))
    # emb = Embedding(output_dim=embedding_vecor_length, input_dim=embedding_size, input_length=seq_size)(nlp_input) 
    # nlp_out = Bidirectional(LSTM(128))(emb) 
    # concat = concatenate([nlp_out, meta_input]) 
    # regressor = Dense(32, activation='relu')(concat) 
    # output = Dense(1, activation='relu')(regressor) 
    # self.model = Model(inputs=[nlp_input , meta_input], outputs=[output])
    # lr_schedule = ExponentialDecay(
    #         lr,
    #         decay_steps=1874,
    #         decay_rate=0.9,
    #         staircase=True)

    # opt = Adam(learning_rate=lr_schedule)
    # self.model.compile(loss="mean_squared_error", optimizer=opt, metrics=['mse'])
    # self.early_stop = EarlyStopping(monitor='loss', min_delta=0.005, patience=5)
    # self.check_pt = ModelCheckpoint(filepath=save_path, save_weights_only=True,
    #                                          verbose=1)

    # # 0.83
    # nlp_input = Input(shape=(seq_size,)) 
    # meta_input = Input(shape=(meta_size,))
    # emb = Embedding(output_dim=embedding_vecor_length, input_dim=embedding_size, input_length=seq_size)(nlp_input) 
    # nlp_out = Bidirectional(LSTM(128))(emb) 
    # concat = concatenate([nlp_out, meta_input]) 
    # regressor = Dense(32, activation='relu')(concat) 
    # output = Dense(1, activation='relu')(regressor) 
    # self.model = Model(inputs=[nlp_input , meta_input], outputs=[output])
    # lr_schedule = ExponentialDecay(
    #         lr,
    #         decay_steps=10000,
    #         decay_rate=0.96,
    #         staircase=True)

    # opt = Adam(learning_rate=lr_schedule)
    # self.model.compile(loss="mean_squared_error", optimizer=opt, metrics=['mse'])

    # self.model.add(Embedding(embedding_size, embedding_vecor_length, input_length=self.input_size))
    # # self.model.add(Dense(128, input_shape=(input_size,), activation='relu'))
    # self.model.add(Dropout(0.2))
    # self.model.add(LSTM(10))
    # self.model.add(Dropout(0.2))
    # self.model.add(Dense(1, activation='relu'))
    # opt = Adam(learning_rate=lr)
    # self.model.compile(loss="mean_squared_error", optimizer=opt, metrics=[])

  def fit(self, seq, meta, y):
    self.model.fit([seq, meta], y, epochs=self.epoch, batch_size=self.batch_size,
                   callbacks=[self.early_stop, self.check_pt])

  def predict(self, seq, meta):
    # seq = seq.reshape(-1, self.seq_size)
    # meta = seq.reshape(-1, self.meta_size)
    return self.model.predict([seq, meta])

In [53]:
num_word = 500
# save_path = "/content/drive/MyDrive/Colab Notebooks/lr001.ckpt"
save_path = "/content/drive/MyDrive/Colab Notebooks/lr001_noTag.ckpt"

In [54]:
def train(X_train, y_train, num_word, save_path, lr=0.01, epoch=100, embedding_size=3000, batch_size=64):
  X_txt_train = X_train[:, :num_word]
  # model = ReviewTextRegressor(num_word, 2, 1, lr=lr, embedding_size=int(X_txt_train.max()+1), epoch=epoch, batch_size=batch_size)
  model = ReviewTextRegressor(num_word, 2, 1, lr=lr, embedding_size=2000, epoch=epoch, batch_size=batch_size, save_path=save_path)
  model.fit(X_txt_train, X_train[:, num_word:], y_train)
  return model
  

In [ ]:
model = train(X_train, y_train, num_word, save_path)

Epoch 1/100
1874/1874 [==============================] - 111s 58ms/step - loss: 1.4575 - mse: 1.4575

Epoch 00001: saving model to /content/drive/MyDrive/Colab Notebooks/lr001_noTag.ckpt
Epoch 2/100
1193/1874 [==================>...........] - ETA: 39s - loss: 0.8694 - mse: 0.8694

In [ ]:
loaded_model = ReviewTextRegressor(num_word, 2, 1, lr=0.01, embedding_size=2000, epoch=40, batch_size=65, save_path=save_path)
loaded_model.model.load_weights(save_path)

In [ ]:
X_test.shape

In [ ]:
predictions = loaded_model.predict(X_test[:, :num_word], X_test[:, num_word:])

In [ ]:
predictions = predictions.squeeze()

In [ ]:
from keras.losses import mean_squared_error
mean_squared_error(y_test, predictions)

In [ ]:
test_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/preproced_test_df.csv")
test_df = test_df.dropna(subset=['preproced_text'])
tokenized_text_test = tokenizer.texts_to_matrix(test_df['preproced_text'], mode='count')

In [ ]:
test_X = np.concatenate([tokenized_text_test, test_df['polarities'].values.reshape(-1, 1), test_df['subjectivities'].values.reshape(-1, 1)], axis=1)

In [ ]:
test_pred = loaded_model.predict(test_X[:, :num_word], test_X[:, num_word:])